In [1]:
%load_ext dotenv
%dotenv

import os
from langchain_openai import ChatOpenAI
from langchain_together import ChatTogether

USE_CHINESE=True

In [2]:
if not USE_CHINESE:
    # Use Mixtral
    model = ChatOpenAI(
        base_url="https://api.together.xyz/v1",
        api_key=os.environ["KEY_TOGETHERAI"],
        model="mistralai/Mixtral-8x7B-Instruct-v0.1",)
else:
    # Use Qwen
    model = ChatTogether(
        together_api_key=os.environ['KEY_TOGETHERAI'],
        model="Qwen/Qwen1.5-72B-Chat",
    )

In [3]:
%reload_ext dotenv

In [4]:
# https://python.langchain.com/v0.2/docs/tutorials/rag/
from random import randint
from langchain import hub
from langchain_chroma import Chroma
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
# from langchain_openai import OpenAIEmbeddings
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.document_loaders import PyMuPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [5]:
loader = PyMuPDFLoader("data/concisehistory-hk.pdf")
docs = loader.load()

In [6]:
# Let's see what have been ingested
print(f"Number of documents:{len(docs)}")

Number of documents:340


In [7]:
docs[randint(0,len(docs)-1)].__dict__

{'page_content': '第\n六\n章\n新\n香\n港\n1 95 \n憶説，許多暴動者「只是一生受壓迫的單純工人」，而政府卻以\n「非常高\n颳和殖民地式的手段」處理騷亂。在暴動後，曾鈺成在大學內成為人們\n避之惟恐不及的人物，但「在左派圈子卻感受到很多朋友之情、同志之\n誼和同情理解」\n0\n21 其後他當上一所香港左派中文學校的校長，並在1 992\n年創立親北京的民主建港聯盟；其弟曾德成則在香港一家親中報章任職。\n但是，對大部分香港人來説，這場動亂賦予了港府新的聲望和合法\n性。六七暴動似乎證明了香港的生活比大陸好，也顯示任何形式的反英\n運動對香港人都沒有好處。當時是香港大學學生的科大衛寫道：「六七暴\n動是首次公開反抗現政府的行動，但對許多人來説，示威、炸彈和不時\n發生的暴動．證明報上所載有關文化大革命的暴行是事實。」\n22 被迫在中\n國和香港之間做選擇的香港人，大多認同殖民地政權是他們的政府· 同\n時．他們愈來愈視自己為一個特殊群體的成員，既區別於殖民地政府，\n又不同於在中國大陸的同胞，\n受到這場動亂的激發，無論英國和香港都有人要求殖民地政府改善\n勞資關係、培養歸屬感、改善官民溝通，並擴大教育一一部分原因是令\n年輕人不會那麼容易接受政治思想灌輸。本地報章擔心緊急法令會繼續\n實行，並批評勞工處無力處理勞資糾紛。對於被捕左派人士該如何處置\n也成了問題\n許多居民希望對參與放炸彈的人判以極刑，但另一些人認\n為，許多參加暴動的人是年少無知，並受鬥委會領袖誤導，而這些領袖\n卻毫髮無損全身而退。有些被告被定罪後判刑很重。例如，\n一名十四歲\n的男孩因攜帶炸彈被判四年，而一名五十四歲男人因參與沙頭角暴動而\n被判刑五年。\n2 1  \nTsang Yok Sing, "Coming of Age in \'67," i n  Blyth and Wotherspoon, Hong Kong Remember\ns, \n91\\. \n22 \nFaure, Coloniltlism and the Hong Kong Mentality, 75-76 \n',
 'metadata': {'source': 'data/concisehistory-hk.pdf',
  'file_path': 'data/conci

In [8]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(docs)

In [9]:
len(splits)

380

In [10]:
splits[100]

Document(page_content='82 \n香港簡史\n西醫治療；並委任曾習西醫的華人醫生常駐該院。政府遂在1897 年委任\n一名華人為「掌院」，監督東華醫院的環境衛生。華人對政府防疫措施的\n抗拒，似乎證明他們的西方健康衛生概念甚為貧乏，歐籍人和一些華人\n精英因而要求更多學校教授英語。這次鼠疫大流行也再次印證了歐籍人\n的成見-----f也們應盡量遠離華人，愈遠愈好。\n不過，這場鼠疫最終撲滅，並非殖民地政府或西方醫學的勝利。雖\n然殖民地的醫療官員對中醫大加撻伐，指它是經驗主義和江湖騙術，批\n評東華繼續提供中醫治療，但殖民地部反對關閉東華醫院。而且即使到\n了十九世纪末，有半數東華醫院病人選擇接受西醫治療，但最初這些病\n人的死亡率比以中醫治療的病人還要高。印度在1896 至1 900 年也爆發鼠\n疫，很可能是由香港傳入。跟香港的惰況一樣，部分曾受西方教育的印\n度中產階級對強制入院和隔離措施感到不滿。謠啄紛傳的情況也和香港\n並無二致，只不過在印度還流傳會發生大災難和英國統治崩潰的謠言。\n其他謠言包括：接種疫苗會令人馬上死亡；醫院毒殺病人，剖開屍體抽\n取他們的「摩米埃」（生命之油）；連身體健康的人也被迫入院，而且沒\n有人能活着出院。印度殖民政府在印度人大規模反抗後，才同意撤銷或\n修改一些較為高壓的抗疫措施。\n1 8 94 年的改革運動\n到了十九世紀末，在各英國殖民地中，自治程度如此低的就只有香\n港。儘管華人很少質疑這個制度，但許多歐籍人認為，自己有份建設這\n個殖民地，理應在管治方面有更多參與。政制改革協會( Constitutional', metadata={'source': 'data/concisehistory-hk.pdf', 'file_path': 'data/concisehistory-hk.pdf', 'page': 91, 'total_pages': 340, 'format': 'PDF 1.6', 'title': '香港簡史: 從殖民地至特別行政區', 'author': '高馬可 & 林立偉', 'subject': '', 'keywords': 'ISBN: 978-988-8263-20-2', 'creator': 'Pdg2Pic', 'producer': 'Adobe Acro

In [11]:
if not USE_CHINESE:
    hfe = HuggingFaceEmbeddings(model_name="BAAI/bge-base-en-v1.5")
else:
    hfe = HuggingFaceEmbeddings(model_name="BAAI/bge-base-zh-v1.5")
    
vectorstore = Chroma.from_documents(documents=docs, embedding=hfe)

/Users/sim/venv/langchain/lib/python3.12/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
/Users/sim/venv/langchain/lib/python3.12/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [12]:

# Retrieve and generate using the relevant snippets of the blog.
retriever = vectorstore.as_retriever(search_kwargs={'k': 6})
prompt = hub.pull("rlm/rag-prompt")

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

rag_chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | model
    | StrOutputParser()
)

In [13]:
rag_chain.invoke("請介紹一下香港島的歷史，請以繁體中文作答")

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


'香港島在1841年1月25日被英國海軍佔領，根據1842年的《南京條約》被清政府永久割讓給英國，直到1997年7月1日成為中國的特別行政區。在第二次世界大戰期間，香港曾被日本佔領。香港島早期的居民以農民、漁民和海盜為主，随着时间的推移，它發展成為一個繁榮的港口和大都會。'

In [14]:
from langchain.chains import RetrievalQAWithSourcesChain
rag_cited_chain = RetrievalQAWithSourcesChain.from_chain_type(model, retriever=retriever)
rag_cited_chain.invoke({"question": "請介紹一下香港島的歷史，請以繁體中文作答"})

{'question': '請介紹一下香港島的歷史，請以繁體中文作答',
 'answer': ' 香港在1842年《南京條約》後被清朝「永久」割讓給英國，並在1997年7月1日回歸中國，成為中華人民共和國的特別行政區。在回歸之前，香港經歷了英國的殖民統治，最初被視為農業和漁業地區，隨著時間的推移，發展成為一個繁榮的港口和大都市。在殖民地時期，香港與中國大陸的聯繫逐漸增加，居民對香港的認同感也日益增強。在二十世紀，香港經歷了各種政治和社會變革，包括在第二次世界大戰中被日本佔領，以及之後的政制改革和與中國的關係討論，最終在1984年的《中英聯合聲明》中確定了1997年的回歸安排。\n',
 'sources': 'data/concisehistory-hk.pdf'}

In [ ]:
rag_chain.invoke("請簡單介紹一下張保仔在香港歷史中的角色和影響，必須要以繁體中文作答")

In [ ]:
rag_chain.invoke("請詳單講述一下彭定康在香港歷史中的角色和影響，必須要以繁體中文作答")